In [1]:
import urllib

params = 'DRIVER={ODBC Driver 13 for SQL Server};' \
         'SERVER=localhost;' \
         'PORT=1433;' \
         'DATABASE=DbAsftPortal;' \
         'UID=sa;' \
         'PWD=aSj#14776$10036%mM&;'
            
params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine

db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

import pandas as pd

sql = '''
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='DbAsftPortal'
'''

pd.read_sql_query(sql, db)

,TABLE_NAME
0,knox_authtoken
1,tbl_library_book
2,tbl_library_book_author
3,tbl_library_book_publisher
4,tbl_library_book_type
5,tbl_library_loaned_book
6,tbl_meeting_cater_type
7,tbl_meeting_equipment
8,tbl_meeting_request
9,tbl_meeting_room


In [5]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from resturaunt.models import Resturaunt_Day_Meal
from django.db.models import Count, Q, F, Window, Value, IntegerField
from django.db.models.functions import RowNumber
from datetime import date, datetime

currentdate = date.today()
print(Resturaunt_Day_Meal.objects.filter(date__exact=currentdate).annotate(
    selectedNo=Count('ResturauntDayMeal_ResturauntEmployeeDayMeal')).annotate(
        servedNo=Count('ResturauntDayMeal_ResturauntEmployeeDayMeal', filter=Q(ResturauntDayMeal_ResturauntEmployeeDayMeal__served=True))).annotate(
            meal_no=Window(
            expression=RowNumber(),
            order_by=F('id').asc())
        ))
# print(Resturaunt_Day_Meal.objects.filter(date__exact=currentdate).annotate(
#     selectedNo=Count('ResturauntDayMeal_ResturauntEmployeeDayMeal')).annotate(
#         servedNo=Count('ResturauntDayMeal_ResturauntEmployeeDayMeal', filter=Q(ResturauntDayMeal_ResturauntEmployeeDayMeal__served=True))).annotate(
#             meal_no=Window(
#             expression=RowNumber(),
#             order_by=F('id').asc())
#         ).order_by('meal_no', 'id'))

NotSupportedError: This backend does not support window expressions.

In [13]:
from resturaunt.models import Resturaunt_Meal, Resturaunt_Day_Meal, Resturaunt_Employee_Day_Meal
from datetime import date, datetime

currentdate = date.today()
departmentId = 2
# date = datetime.strptime(currentdate, '%Y-%m-%d').date()

result1 = Resturaunt_Day_Meal.objects.filter(date__exact=currentdate)
print(result1)
result = Resturaunt_Employee_Day_Meal.objects.filter(resturaunt_day_meal__in=result1, employee__department_id__exact=departmentId).values(
                    'employee__first_name', 'employee__last_name', 'resturaunt_day_meal__resturaunt_meal__name')
print(result)

<QuerySet [<Resturaunt_Day_Meal: Resturaunt_Day_Meal object (108)>, <Resturaunt_Day_Meal: Resturaunt_Day_Meal object (109)>, <Resturaunt_Day_Meal: Resturaunt_Day_Meal object (110)>, <Resturaunt_Day_Meal: Resturaunt_Day_Meal object (111)>]>
<QuerySet [{'employee__first_name': 'علی', 'employee__last_name': 'سجادیان', 'resturaunt_day_meal__resturaunt_meal__name': 'چلو خورش قرمه سبزی'}, {'employee__first_name': 'محسن', 'employee__last_name': 'خزائی', 'resturaunt_day_meal__resturaunt_meal__name': 'چلو خورش قرمه سبزی'}, {'employee__first_name': 'طراح', 'employee__last_name': 'سیستم', 'resturaunt_day_meal__resturaunt_meal__name': 'چلو خورش قرمه سبزی'}, {'employee__first_name': 'علی', 'employee__last_name': 'فتحی', 'resturaunt_day_meal__resturaunt_meal__name': 'چلو خورش قیمه'}, {'employee__first_name': 'علی', 'employee__last_name': 'سجادیان', 'resturaunt_day_meal__resturaunt_meal__name': 'چلو خورش قیمه'}]>
